
### **Project 1 : Natural Language Processing**


**Nama : Fakhrur Novianto**


**Asal : Tangerang**




**Dataset berisi komentar2 mengenai transaksi**

# Preparing Data

In [ ]:
# Import modul yang dibutuhkan

import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Model
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential

# Preprocessing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
# Load Data dari Googledrive

! gdown --id 1AiN_WAoFBCydO1QFEdbLs3tjHrd1WW47

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1AiN_WAoFBCydO1QFEdbLs3tjHrd1WW47
To: /content/complaints_processed.csv
100% 99.5M/99.5M [00:00<00:00, 179MB/s]


In [ ]:
data_awal_1 = pd.read_csv('/content/complaints_processed.csv')
data_awal_1.head()

,Unnamed: 0,product,narrative
0,0,credit_card,purchase order day shipping amount receive pro...
1,1,credit_card,forwarded message date tue subject please inve...
2,2,retail_banking,forwarded message cc sent friday pdt subject f...
3,3,credit_reporting,payment history missing credit report speciali...
4,4,credit_reporting,payment history missing credit report made mis...


# Preprocessing

In [ ]:
# Cek Banyak Kelas

data_awal_1['product'].unique()

array(['credit_card', 'retail_banking', 'credit_reporting',
       'mortgages_and_loans', 'debt_collection'], dtype=object)

In [ ]:
data_awal_1['product'].nunique()

5

In [ ]:
df = data_awal_1.drop(columns = 'Unnamed: 0')
df.head()

,product,narrative
0,credit_card,purchase order day shipping amount receive pro...
1,credit_card,forwarded message date tue subject please inve...
2,retail_banking,forwarded message cc sent friday pdt subject f...
3,credit_reporting,payment history missing credit report speciali...
4,credit_reporting,payment history missing credit report made mis...


In [ ]:
df.nunique()

product           5
narrative    124472
dtype: int64

In [ ]:
# Proses Membersihkan data dengan label 'name' atau tanpa penjelasan narrative ------ Pindahan

df = df[df['narrative'] != 'name']
df

,product,narrative
0,credit_card,purchase order day shipping amount receive pro...
1,credit_card,forwarded message date tue subject please inve...
2,retail_banking,forwarded message cc sent friday pdt subject f...
3,credit_reporting,payment history missing credit report speciali...
4,credit_reporting,payment history missing credit report made mis...
...,...,...
162410,credit_reporting,zales comenity bank closed sold account report...
162411,retail_banking,zelle suspended account without cause banking ...
162412,debt_collection,zero contact made debt supposedly resolved fou...
162413,mortgages_and_loans,zillow home loan nmls nmls actual quote provid...


In [ ]:
df.nunique()

product           5
narrative    124471
dtype: int64

In [ ]:
# Pindahan

df = df.drop_duplicates(subset = ['narrative'], keep = 'first')
df

,product,narrative
0,credit_card,purchase order day shipping amount receive pro...
1,credit_card,forwarded message date tue subject please inve...
2,retail_banking,forwarded message cc sent friday pdt subject f...
3,credit_reporting,payment history missing credit report speciali...
4,credit_reporting,payment history missing credit report made mis...
...,...,...
162410,credit_reporting,zales comenity bank closed sold account report...
162411,retail_banking,zelle suspended account without cause banking ...
162412,debt_collection,zero contact made debt supposedly resolved fou...
162413,mortgages_and_loans,zillow home loan nmls nmls actual quote provid...


In [ ]:
df.nunique()

product           5
narrative    124471
dtype: int64

In [ ]:
# Menghapus baris dengan kolom kosong --------------------- Pindahan

df.dropna(inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
df

,product,narrative
0,credit_card,purchase order day shipping amount receive pro...
1,credit_card,forwarded message date tue subject please inve...
2,retail_banking,forwarded message cc sent friday pdt subject f...
3,credit_reporting,payment history missing credit report speciali...
4,credit_reporting,payment history missing credit report made mis...
...,...,...
162410,credit_reporting,zales comenity bank closed sold account report...
162411,retail_banking,zelle suspended account without cause banking ...
162412,debt_collection,zero contact made debt supposedly resolved fou...
162413,mortgages_and_loans,zillow home loan nmls nmls actual quote provid...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124471 entries, 0 to 162414
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   product    124471 non-null  object
 1   narrative  124471 non-null  object
dtypes: object(2)
memory usage: 2.8+ MB


In [ ]:
# Cek jumlah setiap kelas

df.groupby('product')['product'].count()

product
credit_card            14983
credit_reporting       56240
debt_collection        21056
mortgages_and_loans    18723
retail_banking         13469
Name: product, dtype: int64

In [ ]:
# Ngambil data credit card

df_transaksi_1 = df.loc[df['product'] == 'credit_card']
df_transaksi_1['product'].count()

14983

In [ ]:
# Ngambil data credit reporting

df_transaksi_2 = df.loc[df['product'] == 'credit_reporting']
df_transaksi_2['product'].count()

56240

In [ ]:
# Ngambil data debt collection

df_transaksi_3 = df.loc[df['product'] == 'debt_collection']
df_transaksi_3['product'].count()

21056

In [ ]:
# Ngambil data mortgages and loans

df_transaksi_4 = df.loc[df['product'] == 'mortgages_and_loans']
df_transaksi_4['product'].count()

18723

In [ ]:
# Ngambil data retail banking

df_transaksi_5 = df.loc[df['product'] == 'retail_banking']
df_transaksi_5['product'].count()

13469

In [ ]:
df_transaksi_1_new = df_transaksi_1[0:10000]
df_transaksi_1_new['product'].count()

10000

In [ ]:
df_transaksi_2_new = df_transaksi_2[0:10000]
df_transaksi_2_new['product'].count()

10000

In [ ]:
df_transaksi_3_new = df_transaksi_3[0:10000]
df_transaksi_3_new['product'].count()

10000

In [ ]:
df_transaksi_4_new = df_transaksi_4[0:10000]
df_transaksi_4_new['product'].count()

10000

In [ ]:
df_transaksi_5_new = df_transaksi_5[0:10000]
df_transaksi_5_new['product'].count()

10000

In [ ]:
# Data Fix setelah disamain jumlahnya

df_gabung = [df_transaksi_1_new, df_transaksi_2_new, df_transaksi_3_new, df_transaksi_4_new, df_transaksi_5_new]

df_fix = pd.concat(df_gabung)
df_fix

,product,narrative
0,credit_card,purchase order day shipping amount receive pro...
1,credit_card,forwarded message date tue subject please inve...
11,credit_card,called request new york state covid relief pla...
12,credit_card,capital one secured credit account opened name...
47,credit_card,issue concern discover credit card security de...
...,...,...
114921,retail_banking,received phone call chase service department n...
114926,retail_banking,bank america gave day credit expire bank acct ...
114928,retail_banking,bank america overdrafted account charging doll...
114930,retail_banking,entering citibank online banking credential no...


In [ ]:
df_fix.groupby('product')['product'].count()

product
credit_card            10000
credit_reporting       10000
debt_collection        10000
mortgages_and_loans    10000
retail_banking         10000
Name: product, dtype: int64

In [ ]:
# Baru update sampe sini-----------------------------------------------------------------

In [ ]:
#----------------------------------------------------------------------------------------

In [ ]:
#----------------------------------------------------------------------------------------

In [ ]:
# Proses One Hot Encoding

tipe_transaksi = pd.get_dummies(df_fix['product'])

In [ ]:
tipe_transaksi

,credit_card,credit_reporting,debt_collection,mortgages_and_loans,retail_banking
0,1,0,0,0,0
1,1,0,0,0,0
11,1,0,0,0,0
12,1,0,0,0,0
47,1,0,0,0,0
...,...,...,...,...,...
114921,0,0,0,0,1
114926,0,0,0,0,1
114928,0,0,0,0,1
114930,0,0,0,0,1


In [ ]:
data_proses = pd.concat([df_fix, tipe_transaksi], axis =1)

In [ ]:
data_proses

,product,narrative,credit_card,credit_reporting,debt_collection,mortgages_and_loans,retail_banking
0,credit_card,purchase order day shipping amount receive pro...,1,0,0,0,0
1,credit_card,forwarded message date tue subject please inve...,1,0,0,0,0
11,credit_card,called request new york state covid relief pla...,1,0,0,0,0
12,credit_card,capital one secured credit account opened name...,1,0,0,0,0
47,credit_card,issue concern discover credit card security de...,1,0,0,0,0
...,...,...,...,...,...,...,...
114921,retail_banking,received phone call chase service department n...,0,0,0,0,1
114926,retail_banking,bank america gave day credit expire bank acct ...,0,0,0,0,1
114928,retail_banking,bank america overdrafted account charging doll...,0,0,0,0,1
114930,retail_banking,entering citibank online banking credential no...,0,0,0,0,1


In [ ]:
data_proses = data_proses.drop(columns = 'product')

In [ ]:
data_proses

,narrative,credit_card,credit_reporting,debt_collection,mortgages_and_loans,retail_banking
0,purchase order day shipping amount receive pro...,1,0,0,0,0
1,forwarded message date tue subject please inve...,1,0,0,0,0
11,called request new york state covid relief pla...,1,0,0,0,0
12,capital one secured credit account opened name...,1,0,0,0,0
47,issue concern discover credit card security de...,1,0,0,0,0
...,...,...,...,...,...,...
114921,received phone call chase service department n...,0,0,0,0,1
114926,bank america gave day credit expire bank acct ...,0,0,0,0,1
114928,bank america overdrafted account charging doll...,0,0,0,0,1
114930,entering citibank online banking credential no...,0,0,0,0,1


In [ ]:
# Stemming

#from nltk.tokenize import word_tokenize
#tokens = word_tokenize(text)

#from nltk.stem.porter import PorterStemmer
#porter = PorterStemmer()
#stemmed = [porter.stem(word) for word in tokens]

In [ ]:
# Stopwords

import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_word = set(stopwords.words('english'))
data_proses['narrative'] = data_proses['narrative'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_word)]))
data_proses.head()


,narrative,credit_card,credit_reporting,debt_collection,mortgages_and_loans,retail_banking
0,purchase order day shipping amount receive pro...,1,0,0,0,0
1,forwarded message date tue subject please inve...,1,0,0,0,0
11,called request new york state covid relief pla...,1,0,0,0,0
12,capital one secured credit account opened name...,1,0,0,0,0
47,issue concern discover credit card security de...,1,0,0,0,0


In [ ]:
len(data_proses['narrative'][0])

1705

In [ ]:
type(data_proses['narrative'])

pandas.core.series.Series

In [ ]:
# Proses Membersihkan data dengan label 'name' atau tanpa penjelasan narrative

#data_proses = data_proses[data_proses['narrative'] != 'name']

In [ ]:
#data_proses

In [ ]:
#data_proses = data_proses.drop_duplicates(subset = ['narrative'], keep = 'first')
#data_proses

In [ ]:
#data_proses.info()

In [ ]:
#data_proses['narrative'].nunique()

In [ ]:
# Menghapus baris dengan kolom kosong

#data_proses.dropna(inplace = True)

In [ ]:
#data_proses.info()

In [ ]:
#data_proses['narrative'].nunique( )

In [ ]:
print(data_proses['narrative'].values)

['purchase order day shipping amount receive product week sent followup email exact verbiage paid two day shipping received order company responded im sorry inform due unusually high order volume order shipped several week stock since early due high demand although continuing take order guaranteeing receive order place due time mask order exact shipping date right however guarantee ship soon soon delivers product u getting small shipment shipping first come first served basis appreciate patience fulfill order quickly recommend keeping order lose place line cancel distributor stock moment prefer cancel please note ask via email cancel accordance cancellation policy agreed checkout electronic inventory online requested order canceled refund issued canceled order sent verification order canceled refunded item particulate respirator refunded subtotal shipping tax total usd visa ending refund called disputed amount stated nothing needed submitted address issue recharged item removing called

In [ ]:
# Jumlah Kata

from collections import Counter

def wordCount(narrative):
  count = Counter()
  for i in narrative.values:
    for word in i.split():
      count[word] +=1
  return count

narrative = data_proses['narrative']
counter = wordCount(narrative)


jumlah_kata = len(counter)
print(jumlah_kata)

32220


# Processing

In [ ]:
# Mengubah dataframe menjadi array

komentar = data_proses['narrative'].values
jenis = data_proses[['credit_card','credit_reporting','debt_collection','mortgages_and_loans','retail_banking']].values

In [ ]:
# Membagi data latih dan data tes

komentar_latih, komentar_tes, jenis_latih, jenis_tes = train_test_split(komentar, jenis, test_size = 0.2, random_state = None)

In [ ]:
len(komentar_latih), len(komentar_tes)

(40000, 10000)

In [ ]:
len(jenis_latih), len(jenis_tes)

(40000, 10000)

In [ ]:
# Proses tokenisasi
#max_length = 2685 #coba ubah 1705 length minimal di data set, 20596 length maksimal di dataset

num_words = jumlah_kata

# 1 minimal jumlah kata, 2685 maksimal jumlah kata

tokenizer = Tokenizer(num_words, oov_token = '-', lower = True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', split=' ') #num word coba ubah
tokenizer.fit_on_texts(komentar_latih)
#tokenizer.fit_on_texts(komentar_tes)

# Toeknisasi pada kalimat
sekuen_latih = tokenizer.texts_to_sequences(komentar_latih)
sekuen_tes = tokenizer.texts_to_sequences(komentar_tes)


max_length = max([len(x) for x in sekuen_latih])
print(max_length)

# Proses penyamaan panjang kalimat dengan mengisi kata yang kosong dengan 0
pengisi_latih = pad_sequences(sekuen_latih, maxlen= max_length, truncating= 'post', padding = 'post')
pengisi_tes = pad_sequences(sekuen_tes, maxlen = max_length, truncating= 'post', padding = 'post') # belum paham truncating sama pading di tes perlu apa ngga

2568


In [ ]:
# Bikin model

# Otak atik disini

# Tambahan ReduceLROnPlateau Callback ----- nanti dimasukin ke model.fit -----Masih nyoba

from keras.callbacks import ReduceLROnPlateau

rlop= ReduceLROnPlateau(monitor = 'val_loss', 
                        factor = 0.1, 
                        patience = 10)



from keras.layers import Dropout, Bidirectional # Coba bidirectional

model = Sequential([Embedding(input_dim = num_words, output_dim = 64, input_length= max_length),
                    Dropout(0.2),
                    Bidirectional(LSTM(64, return_sequences = True)), # masih nyoba
                    Dropout(0.2),
                    #Bidirectional(LSTM(64)), # masih nyoba
                    #Dropout(0.2),
                    #Dense(256, activation = 'relu'), # Masih trail jumlah unit
                    #Dropout(0.2),
                    #Dense(64, activation = 'relu'),
                    #Dropout(0.3),
                    #Dense(64, activation = 'relu'),
                    #Dropout(0.5),
                    #Dense(32, activation = 'relu'),
                    #Dropout(0.2),
                    Dense(5, activation = 'softmax')])


from tensorflow.keras.optimizers import Adam # Coba---------------------------

b2 = 0.99
ad_lr = Adam(learning_rate = 0.001, 
             beta_2=b2
             )

model.compile(#optimizer = 'Adam',
              optimizer = ad_lr,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
# Val_loss masih tinggi karena overfitting

# jadi harus pake layer Dropout


jumlah_epoch = 10 # Masih trail jumlah epoch
hist = model.fit(pengisi_latih,
                 jenis_latih,
                 epochs = jumlah_epoch,
                 batch_size = 8,
                 validation_data = (pengisi_tes, jenis_tes),
                 validation_split = 0.2,
                 verbose = 2,
                 callbacks = [rlop]
                 )

Epoch 1/10


KeyboardInterrupt: ignored

In [ ]:
# Plot Akurasi dan Loss

import matplotlib.pyplot as plt

plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.ylabel('Akurasi')
plt.xlabel('Epoch')
plt.legend()
plt.title('Train Acc vs Validation Acc')

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.title('Train Loss vs Validation Loss')